In [ ]:
# 2D for bert embedding, elmo embedding and so on

In [1]:
# !pip install git+https://www.github.com/keras-team/keras-contrib.git

In [2]:
import numpy as np 
import pandas as pd 
import keras
print(keras.__version__)

from math import nan
from keras.callbacks import ModelCheckpoint

from keras_contrib.layers import CRF



/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/framew

2.3.1


/root/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dty

In [3]:
dframe = pd.read_csv("ner_small.csv", encoding = "ISO-8859-1", error_bad_lines=False)

In [4]:
dataset=dframe.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word',"pos"],axis=1)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15591 entries, 0 to 15590
Data columns (total 4 columns):
sentence_idx    15591 non-null int64
shape           15591 non-null object
word            15591 non-null object
tag             15591 non-null object
dtypes: int64(1), object(3)
memory usage: 487.3+ KB


In [6]:
dataset.head()

,sentence_idx,shape,word,tag
0,1,capitalized,Thousands,O
1,1,lowercase,of,O
2,1,lowercase,demonstrators,O
3,1,lowercase,have,O
4,1,lowercase,marched,O


In [7]:
dataset=dataset.drop(['shape'],axis=1)

In [8]:
dataset.head()

,sentence_idx,word,tag
0,1,Thousands,O
1,1,of,O
2,1,demonstrators,O
3,1,have,O
4,1,marched,O


In [9]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["word"].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [10]:
getter = SentenceGetter(dataset)

In [11]:
sentences = getter.sentences

In [12]:
print(sentences[5])

[('The', 'O'), ('party', 'O'), ('is', 'O'), ('divided', 'O'), ('over', 'O'), ('Britain', 'B-gpe'), ("'s", 'O'), ('participation', 'O'), ('in', 'O'), ('the', 'O'), ('Iraq', 'B-geo'), ('conflict', 'O'), ('and', 'O'), ('the', 'O'), ('continued', 'O'), ('deployment', 'O'), ('of', 'O'), ('8,500', 'O'), ('British', 'B-gpe'), ('troops', 'O'), ('in', 'O'), ('that', 'O'), ('country', 'O'), ('.', 'O')]


In [13]:
maxlen = max([len(s) for s in sentences])
print ('Maximum sequence length:', maxlen)

Maximum sequence length: 62


In [14]:
words = list(set(dataset["word"].values))
words.append("ENDPAD")

In [15]:
n_words = len(words); n_words

3691

In [16]:
tags = []
for tag in set(dataset["tag"].values):
    if tag is nan or isinstance(tag, float):
        tags.append('unk')
    else:
        tags.append(tag)
print(tags)

['I-per', 'I-gpe', 'I-nat', 'B-geo', 'I-art', 'I-org', 'I-eve', 'B-eve', 'O', 'B-per', 'B-gpe', 'B-tim', 'B-art', 'I-geo', 'B-org', 'I-tim', 'B-nat']


In [17]:
n_tags = len(tags); n_tags

17

In [18]:
from future.utils import iteritems
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}

In [19]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [20]:
np.array(X).shape
X2=X

In [21]:
X = pad_sequences(maxlen=1400, sequences=X, padding="post",value=n_words - 1)
X2 = pad_sequences(maxlen=140, sequences=X2, padding="post",value=n_words - 1)

In [22]:
y_idx = [[tag2idx[w[1]] for w in s] for s in sentences]
print(sentences[100])
print(y_idx[100])

[('The', 'O'), ('Pakistani', 'B-gpe'), ('military', 'O'), ('launched', 'O'), ('its', 'O'), ('offensive', 'O'), ('in', 'O'), ('Orakzai', 'B-geo'), ('to', 'O'), ('hunt', 'O'), ('Taliban', 'B-org'), ('insurgents', 'O'), ('.', 'O')]
[8, 10, 8, 8, 8, 8, 8, 3, 8, 8, 14, 8, 8]


In [23]:
y = pad_sequences(maxlen=140, sequences=y_idx, padding="post", value=tag2idx["O"])
print(y_idx[100])

[8, 10, 8, 8, 8, 8, 8, 3, 8, 8, 14, 8, 8]


In [24]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [25]:
X.shape

(702, 1400)

In [26]:
X = X.reshape((X.shape[0],140,10))
X.shape

(702, 140, 10)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size=0.2, random_state=666)

In [28]:
X_train.shape

(561, 140, 10)

In [29]:
X_train[0]

array([[1452, 2209,  762, ..., 2749,  678, 2901],
       [1778, 2352, 3613, ..., 3690, 3690, 3690],
       [3690, 3690, 3690, ..., 3690, 3690, 3690],
       ...,
       [3690, 3690, 3690, ..., 3690, 3690, 3690],
       [3690, 3690, 3690, ..., 3690, 3690, 3690],
       [3690, 3690, 3690, ..., 3690, 3690, 3690]], dtype=int32)

In [30]:
y_train[1],y_train[1].shape

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), (140, 17))

In [31]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k


In [32]:
input = Input(shape=(140,10))
word_embedding_size = 10
model = Bidirectional(LSTM(units=word_embedding_size, 
                           return_sequences=True, 
                           dropout=0.5, 
                           recurrent_dropout=0.5, 
                           kernel_initializer=k.initializers.he_normal()))(input)
model = LSTM(units=word_embedding_size * 2, 
             return_sequences=True, 
             dropout=0.5, 
             recurrent_dropout=0.5, 
             kernel_initializer=k.initializers.he_normal())(model)
model = TimeDistributed(Dense(n_tags, activation="relu"))(model)  # previously softmax output layer

crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output


In [33]:
model = Model(input, out)


In [34]:
adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)
#model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizer=adam, loss=crf.loss_function, metrics=[crf.accuracy, 'accuracy'])


/root/anaconda3/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/root/anaconda3/lib/python3.6/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [35]:
history = model.fit(X_train, np.array(y_train), batch_size=256, epochs=3, validation_split=0.2, verbose=1)



Train on 448 samples, validate on 113 samples
Epoch 1/3
448/448 [==============================] - 3s 6ms/step - loss: 2.7265 - crf_viterbi_accuracy: 0.1537 - accuracy: 0.0028 - val_loss: 2.7363 - val_crf_viterbi_accuracy: 0.0044 - val_accuracy: 0.0044
Epoch 2/3
448/448 [==============================] - 1s 2ms/step - loss: 2.7060 - crf_viterbi_accuracy: 0.1636 - accuracy: 0.0028 - val_loss: 2.7054 - val_crf_viterbi_accuracy: 0.0052 - val_accuracy: 0.0052
Epoch 3/3
448/448 [==============================] - 1s 2ms/step - loss: 2.6905 - crf_viterbi_accuracy: 0.1517 - accuracy: 0.0028 - val_loss: 2.6764 - val_crf_viterbi_accuracy: 0.0068 - val_accuracy: 0.0068


In [36]:
p = model.predict(np.array([X_test[0]]))
p = np.argmax(p, axis=-1)
print(p)

[[ 5  1  9 10  3  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9
   1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9
   1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9
   1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9
   1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9
   1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9  1  9]]


In [37]:
X_test2.shape

(141, 140)

In [38]:
gt = np.argmax(y_test[0], axis=-1)
print(gt)
print("{:14}: ({:5}): {}".format("Word", "True", "Pred"))
for idx, (w,pred) in enumerate(zip(X_test2[0],p[0])):
    #
    print("{:14}: ({:5}): {}".format(words[w],idx2tag[gt[idx]],tags[pred]))


[8 8 8 8 8 9 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
Word          : (True ): Pred
A             : (O    ): I-org
military      : (O    ): I-gpe
tribunal      : (O    ): B-per
has           : (O    ): B-gpe
charged       : (O    ): B-geo
Dr.           : (B-per): B-per
Besigye       : (I-per): I-gpe
with          : (O    ): B-per
terrorism     : (O    ): I-gpe
and           : (O    ): B-per
possessing    : (O    ): I-gpe
illegal       : (O    ): B-per
weapons       : (O    ): I-gpe
.             : (O    ): B-per
ENDPAD        : (O    ): I-gpe
ENDPAD        : (O    ): B-per
ENDPAD        : (O    ): I-gpe
ENDPAD        : (O    ): B-per
ENDPAD        : (O    ): I-gpe
ENDPAD        : (O    ): B-per
ENDPAD        : (O    ): I-gpe
ENDPAD        : (O    ): B-per
END